# PyTorch Contest Toolkit
Полный набор шаблонов и утилит для решения задач **NLP, CV и табличного ML** на олимпиадных конкурсах.
*Версия:* 2025-05-06


## Содержание
1. [Установка окружения](#setup)
2. [Общие утилиты](#utils)
3. [Базовый тренировочный движок](#engine)
4. [Computer Vision](#cv)
5. [Natural Language Processing](#nlp)
6. [Tabular / General ML](#ml)
7. [Сохранение и загрузка моделей](#ckpt)
8. [Дополнительные приёмы](#extras)


## 1. Установка окружения <a id='setup'></a>

In [ ]:
# Если работаете в окружении без PyTorch:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# Другие полезные пакеты (по необходимости):
# !pip install transformers sentencepiece scikit-learn numpy pandas matplotlib tqdm albumentations


## 2. Общие утилиты <a id='utils'></a>

In [ ]:
import random, os, math, time, json, copy, gc, logging, sys
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def get_device():
    return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

set_seed()
device = get_device()
print('Using device:', device)


## 3. Базовый тренировочный движок <a id='engine'></a>
Единый цикл обучения и валидации, применимый к любому типу задачи.

In [ ]:
class Trainer:
    def __init__(self, model, criterion, optimizer, scheduler=None,
                 mixed_precision=True, grad_clip=None):
        self.model = model.to(device)
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.scaler = torch.cuda.amp.GradScaler(enabled=mixed_precision)
        self.grad_clip = grad_clip

    def _step(self, batch, train=True):
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)
        with torch.cuda.amp.autocast(enabled=self.scaler.is_enabled()):
            outputs = self.model(inputs)
            loss = self.criterion(outputs, targets)
        if train:
            self.optimizer.zero_grad()
            self.scaler.scale(loss).backward()
            if self.grad_clip:
                self.scaler.unscale_(self.optimizer)
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.grad_clip)
            self.scaler.step(self.optimizer)
            self.scaler.update()
        return loss.item(), outputs.detach().cpu(), targets.detach().cpu()

    def loop(self, loader, train=True):
        self.model.train() if train else self.model.eval()
        epoch_loss, preds, gts = 0.0, [], []
        with torch.set_grad_enabled(train):
            for batch in tqdm(loader, leave=False):
                loss, out, tgt = self._step(batch, train)
                epoch_loss += loss * len(tgt)
                preds.append(out)
                gts.append(tgt)
        preds = torch.cat(preds)
        gts = torch.cat(gts)
        return epoch_loss / len(loader.dataset), preds, gts

    def fit(self, train_loader, val_loader=None, epochs=10,
            metric_fn=None, ckpt_path=None, early_stop=None):
        best_metric, patience = None, 0
        for epoch in range(1, epochs+1):
            tr_loss, *_ = self.loop(train_loader, train=True)
            if self.scheduler: self.scheduler.step()
            if val_loader:
                val_loss, preds, gts = self.loop(val_loader, train=False)
                metric_val = metric_fn(preds, gts) if metric_fn else val_loss
                print(f'Epoch {epoch} | train: {tr_loss:.4f} | val: {val_loss:.4f} | metric: {metric_val:.4f}')
                if best_metric is None or metric_val > best_metric:
                    best_metric, patience = metric_val, 0
                    if ckpt_path:
                        torch.save(self.model.state_dict(), ckpt_path)
                        print('  ✔ Новый лучший чекпоинт сохранён')
                else:
                    patience += 1
                    if early_stop and patience >= early_stop:
                        print('Early stopping!')
                        break
            else:
                print(f'Epoch {epoch} | loss: {tr_loss:.4f}')


## 4. Computer Vision <a id='cv'></a>
Примеры датасета, аугментаций, модели и запуска обучения.

In [ ]:
from torchvision import transforms, datasets, models

# ---- Dataset & transforms ----
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
val_tfms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

# Example using ImageFolder (понадобится структура данных class_name/xxx.jpg)
# train_ds = datasets.ImageFolder('train_images', transform=train_tfms)
# val_ds   = datasets.ImageFolder('val_images', transform=val_tfms)

# ---- Model ----
class SmallCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.classifier = nn.Linear(64*56*56, num_classes)
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

# resnet = models.resnet18(weights=None)  # разрешено — без предобученных весов
# resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

def accuracy(preds, gts):
    return (preds.argmax(dim=1) == gts).float().mean().item()

# ---- Training example (commented) ----
# batch_size = 32
# train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)
# val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=4)
# model = SmallCNN(num_classes=len(train_ds.classes))
# trainer = Trainer(model, nn.CrossEntropyLoss(),
#                   torch.optim.Adam(model.parameters(), lr=1e-3),
#                   scheduler=None, mixed_precision=True, grad_clip=1.0)
# trainer.fit(train_loader, val_loader, epochs=10, metric_fn=accuracy,
#             ckpt_path='best_cnn.pth', early_stop=3)


## 5. Natural Language Processing <a id='nlp'></a>
Шаблон для классификации текста с Embedding + LSTM.
Для токенизации можно использовать `torchtext`, `sentencepiece`, или собственный словарь.

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# ---- Vocabulary ----
class Vocab:
    def __init__(self, tokens, min_freq=2):
        from collections import Counter
        counter = Counter(tokens)
        self.itos = ['<pad>', '<unk>'] + [t for t,c in counter.items() if c >= min_freq]
        self.stoi = {t:i for i,t in enumerate(self.itos)}
    def encode(self, tokens):
        return [self.stoi.get(t, 1) for t in tokens]

# ---- Dataset ----
class TextDataset(Dataset):
    def __init__(self, texts, labels, vocab=None):
        self.raw_texts = [t.split() for t in texts]
        self.labels = torch.tensor(labels)
        if vocab is None:
            vocab = Vocab([tok for txt in self.raw_texts for tok in txt])
        self.vocab = vocab
    def __len__(self): return len(self.labels)
    def __getitem__(self, idx):
        return torch.tensor(self.vocab.encode(self.raw_texts[idx])), self.labels[idx]

def collate_fn(batch):
    seqs, labels = zip(*batch)
    seqs_padded = pad_sequence(seqs, batch_first=True, padding_value=0)
    return seqs_padded, torch.tensor(labels)

# ---- Model ----
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):
        x = self.embedding(x)
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1])

# ---- Training example (commented) ----
# vocab = Vocab([tok for txt in train_texts for tok in txt.split()])
# train_ds = TextDataset(train_texts, train_labels, vocab)
# val_ds   = TextDataset(val_texts, val_labels, vocab)
# train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn)
# val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, collate_fn=collate_fn)
# model = LSTMClassifier(len(vocab.itos), 128, 256, num_classes=2)
# trainer = Trainer(model, nn.CrossEntropyLoss(),
#                   torch.optim.Adam(model.parameters(), lr=2e-3),
#                   mixed_precision=False)
# trainer.fit(train_loader, val_loader, epochs=6, metric_fn=accuracy, ckpt_path='best_lstm.pth')


## 6. Табличный / общий ML <a id='ml'></a>
MLP для регрессии/классификации с числовыми признаками.

In [ ]:
class MLP(nn.Module):
    def __init__(self, in_dim, hidden_layers=[256,128], out_dim=1, drop_p=0.2):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden_layers:
            layers += [nn.Linear(last, h), nn.BatchNorm1d(h), nn.ReLU(), nn.Dropout(drop_p)]
            last = h
        layers.append(nn.Linear(last, out_dim))
        self.net = nn.Sequential(*layers)
    def forward(self, x): return self.net(x)

# Dataset skeleton
class TabDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# Metric examples
def rmse(preds, gts): return ((preds.squeeze() - gts.float())**2).mean().sqrt().item()
def acc_cls(preds, gts): return (preds.argmax(1) == gts).float().mean().item()


## 7. Сохранение / загрузка моделей <a id='ckpt'></a>

In [ ]:
def save_ckpt(model, path, extras=None):
    torch.save({'state_dict': model.state_dict(),
                'extras': extras or {}}, path)

def load_ckpt(model, path, map_location=None):
    chkpt = torch.load(path, map_location=map_location or device)
    model.load_state_dict(chkpt['state_dict'])
    return chkpt.get('extras', {})


## 8. Дополнительные приёмы <a id='extras'></a>
- **Grad Accumulation**: аккумулируйте градиенты для больших батчей.
- **CosineAnnealingLR / OneCycleLR**: гибкое расписание обучения.
- **K-Fold Cross-Validation**: усреднение моделей.
- **Snapshot Ensembling**: сохраняйте несколько чекпоинтов.
- **Mixed Precision (`torch.cuda.amp`)**: ускоряет и экономит память на GPU.
- **Profiling**: `torch.profiler`, `nvprof`, `nsys`.
- **Debugging**: `detect_anomaly`, `torch.autograd.set_detect_anomaly(True)`.
